<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Stable_diffusion_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Stable diffusion implementation on Google Colab**

Official GitHub --> https://github.com/CompVis/stable-diffusion

In [1]:
!git clone https://github.com/CompVis/stable-diffusion.git
%cd stable-diffusion

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 313, done.
remote: Total 313 (delta 0), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (313/313), 42.62 MiB | 34.77 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/stable-diffusion


In [ ]:
#####Python3.7で作動
# !sudo add-apt-repository -y ppa:deadsnakes/ppa
# !sudo apt-get -y update
# !sudo apt-get -y install python3.7
# !sudo apt-get -y install python3.7-dev
# !sudo apt-get -y install python3-pip
# !sudo apt-get -y install python3.7-distutils
# !python3.7 -m pip install --upgrade setuptools
# !python3.7 -m pip install --upgrade pip
# !python3.7 -m pip install --upgrade distlib
# !sudo update-alternatives --set python /usr/bin/python3.7
# !sudo ln -sf /usr/bin/python /usr/local/bin/python

modules = """
pip==20.3
torchvision==0.12.0
numpy==1.19.2
albumentations==0.4.3
diffusers
opencv-python==4.1.2.30
pudb==2019.2
imageio==2.9.0
imageio-ffmpeg==0.4.2
omegaconf==2.1.1
test-tube>=0.7.5
streamlit>=0.73.1
einops==0.3.0
torch-fidelity==0.3.0
transformers==4.19.2
torchmetrics==0.6.0
kornia==0.6
invisible-watermark
-e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
-e git+https://github.com/openai/CLIP.git@main#egg=clip
-e .
"""

# modules = """
# pip==20.3
# torch==1.7.1
# torchvision==0.8.2
# albumentations==0.4.3 
# opencv-python==4.6.0.66 
# pudb==2019.2 
# imageio==2.9.0 
# imageio-ffmpeg==0.4.2 
# pytorch-lightning==1.5.0 
# omegaconf==2.1.1 
# test-tube>=0.7.5 
# streamlit>=0.73.1 
# einops==0.3.0 
# torch-fidelity==0.3.0 
# transformers==4.19.2 
# torchmetrics==0.6.0 
# kornia==0.6
# torchtext==0.8.1
# invisible-watermark
# -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers 
# -e git+https://github.com/openai/CLIP.git@main#egg=clip 
# -e .
# """

# modules = """
# pip
# torchvision
# numpy
# albumentations
# diffusers
# opencv-python
# pudb
# invisible-watermark
# imageio
# imageio-ffmpeg
# omegaconf
# test-tube
# streamlit
# einops
# torch-fidelity
# transformers
# torchmetrics
# kornia
# -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
# -e git+https://github.com/openai/CLIP.git@main#egg=clip
# -e .
# """

!pip install --upgrade pip
!pip install --upgrade torchtext 

with open("requirements.txt", mode="w") as f:
    f.write(modules)
!pip install -r requirements.txt

In [3]:
from torchtext import legacy

OSError: ignored

In [17]:
y
import torch
print(torch.__version__)

1.12.1+cu113


In [ ]:
!python scripts/txt2img.py --H 512 --W 512 --seed 27 --n_iter 2 --n_samples 10 --ddim_steps 50 --prompt "Venetian masquerade bell, Dark blue night, mysterious, casanova, romantic, cinematic, light coming through the celling dome. Piazza San"


In [ ]:
import argparse, os, sys, datetime, glob, importlib, csv
import numpy as np
import time
import torch
import torchvision
import pytorch_lightning as pl

from packaging import version
from omegaconf import OmegaConf
from torch.utils.data import random_split, DataLoader, Dataset, Subset
from functools import partial
from PIL import Image

from pytorch_lightning import seed_everything
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, Callback, LearningRateMonitor
from pytorch_lightning.utilities.distributed import rank_zero_only
from pytorch_lightning.utilities import rank_zero_info

from ldm.data.base import Txt2ImgIterableBaseDataset
from ldm.util import instantiate_from_config

import os
import shutil

def makedirs(path):
    if os.path.exists():
        shutil.rmtree(path)
    os.makedirs(path)


In [ ]:
logdir = "./log"
resumedir = "./resume"
seed = 23

sys.path.append(os.getcwd())

# parser = get_parser()
# parser = Trainer.add_argparse_args(parser)
#opt, unknown = parser.parse_known_args()

# if opt.name and opt.resume:
#     raise ValueError(
#         "-n/--name and -r/--resume cannot be specified both."
#         "If you want to resume training in a new log folder, "
#         "use -n/--name in combination with --resume_from_checkpoint"
#     )
# if opt.resume:
#     if not os.path.exists(opt.resume):
#         raise ValueError("Cannot find {}".format(opt.resume))
#     if os.path.isfile(opt.resume):
#         paths = opt.resume.split("/")
#         # idx = len(paths)-paths[::-1].index("logs")+1
#         # logdir = "/".join(paths[:idx])
#         logdir = "/".join(paths[:-2])
#         ckpt = opt.resume
#     else:
#         assert os.path.isdir(opt.resume), opt.resume
#         logdir = opt.resume.rstrip("/")
#         ckpt = os.path.join(logdir, "checkpoints", "last.ckpt")

#     opt.resume_from_checkpoint = ckpt
#     base_configs = sorted(glob.glob(os.path.join(logdir, "configs/*.yaml")))
#     opt.base = base_configs + opt.base
#     _tmp = logdir.split("/")
#     nowname = _tmp[-1]
# else:
#     if opt.name:
#         name = "_" + opt.name
#     elif opt.base:
#         cfg_fname = os.path.split(opt.base[0])[-1]
#         cfg_name = os.path.splitext(cfg_fname)[0]
#         name = "_" + cfg_name
#     else:
#         name = ""
#     nowname = now + name + opt.postfix
#     logdir = os.path.join(opt.logdir, nowname)

ckptdir = os.path.join(logdir, "checkpoints")
cfgdir = os.path.join(logdir, "configs")
seed_everything(seed)

try:
    # init and save configs
    configs = [OmegaConf.load(cfg) for cfg in opt.base]
    cli = OmegaConf.from_dotlist(unknown)
    config = OmegaConf.merge(*configs, cli)
    lightning_config = config.pop("lightning", OmegaConf.create())
    # merge trainer cli with config
    trainer_config = lightning_config.get("trainer", OmegaConf.create())
    # default to ddp
    trainer_config["accelerator"] = "ddp"
    for k in nondefault_trainer_args(opt):
        trainer_config[k] = getattr(opt, k)
    if not "gpus" in trainer_config:
        del trainer_config["accelerator"]
        cpu = True
    else:
        gpuinfo = trainer_config["gpus"]
        print(f"Running on GPUs {gpuinfo}")
        cpu = False
    trainer_opt = argparse.Namespace(**trainer_config)
    lightning_config.trainer = trainer_config

    # model
    model = instantiate_from_config(config.model)

    # trainer and callbacks
    trainer_kwargs = dict()

    # default logger configs
    default_logger_cfgs = {
        "wandb": {
            "target": "pytorch_lightning.loggers.WandbLogger",
            "params": {
                "name": nowname,
                "save_dir": logdir,
                "offline": opt.debug,
                "id": nowname,
            }
        },
        "testtube": {
            "target": "pytorch_lightning.loggers.TestTubeLogger",
            "params": {
                "name": "testtube",
                "save_dir": logdir,
            }
        },
    }
    default_logger_cfg = default_logger_cfgs["testtube"]
    if "logger" in lightning_config:
        logger_cfg = lightning_config.logger
    else:
        logger_cfg = OmegaConf.create()
    logger_cfg = OmegaConf.merge(default_logger_cfg, logger_cfg)
    trainer_kwargs["logger"] = instantiate_from_config(logger_cfg)

    # modelcheckpoint - use TrainResult/EvalResult(checkpoint_on=metric) to
    # specify which metric is used to determine best models
    default_modelckpt_cfg = {
        "target": "pytorch_lightning.callbacks.ModelCheckpoint",
        "params": {
            "dirpath": ckptdir,
            "filename": "{epoch:06}",
            "verbose": True,
            "save_last": True,
        }
    }
    if hasattr(model, "monitor"):
        print(f"Monitoring {model.monitor} as checkpoint metric.")
        default_modelckpt_cfg["params"]["monitor"] = model.monitor
        default_modelckpt_cfg["params"]["save_top_k"] = 3

    if "modelcheckpoint" in lightning_config:
        modelckpt_cfg = lightning_config.modelcheckpoint
    else:
        modelckpt_cfg =  OmegaConf.create()
    modelckpt_cfg = OmegaConf.merge(default_modelckpt_cfg, modelckpt_cfg)
    print(f"Merged modelckpt-cfg: \n{modelckpt_cfg}")
    if version.parse(pl.__version__) < version.parse('1.4.0'):
        trainer_kwargs["checkpoint_callback"] = instantiate_from_config(modelckpt_cfg)

    # add callback which sets up log directory
    default_callbacks_cfg = {
        "setup_callback": {
            "target": "main.SetupCallback",
            "params": {
                "resume": opt.resume,
                "now": now,
                "logdir": logdir,
                "ckptdir": ckptdir,
                "cfgdir": cfgdir,
                "config": config,
                "lightning_config": lightning_config,
            }
        },
        "image_logger": {
            "target": "main.ImageLogger",
            "params": {
                "batch_frequency": 750,
                "max_images": 4,
                "clamp": True
            }
        },
        "learning_rate_logger": {
            "target": "main.LearningRateMonitor",
            "params": {
                "logging_interval": "step",
                # "log_momentum": True
            }
        },
        "cuda_callback": {
            "target": "main.CUDACallback"
        },
    }
    if version.parse(pl.__version__) >= version.parse('1.4.0'):
        default_callbacks_cfg.update({'checkpoint_callback': modelckpt_cfg})

    if "callbacks" in lightning_config:
        callbacks_cfg = lightning_config.callbacks
    else:
        callbacks_cfg = OmegaConf.create()

    if 'metrics_over_trainsteps_checkpoint' in callbacks_cfg:
        print(
            'Caution: Saving checkpoints every n train steps without deleting. This might require some free space.')
        default_metrics_over_trainsteps_ckpt_dict = {
            'metrics_over_trainsteps_checkpoint':
                {"target": 'pytorch_lightning.callbacks.ModelCheckpoint',
                  'params': {
                      "dirpath": os.path.join(ckptdir, 'trainstep_checkpoints'),
                      "filename": "{epoch:06}-{step:09}",
                      "verbose": True,
                      'save_top_k': -1,
                      'every_n_train_steps': 10000,
                      'save_weights_only': True
                  }
                  }
        }
        default_callbacks_cfg.update(default_metrics_over_trainsteps_ckpt_dict)

    callbacks_cfg = OmegaConf.merge(default_callbacks_cfg, callbacks_cfg)
    if 'ignore_keys_callback' in callbacks_cfg and hasattr(trainer_opt, 'resume_from_checkpoint'):
        callbacks_cfg.ignore_keys_callback.params['ckpt_path'] = trainer_opt.resume_from_checkpoint
    elif 'ignore_keys_callback' in callbacks_cfg:
        del callbacks_cfg['ignore_keys_callback']

    trainer_kwargs["callbacks"] = [instantiate_from_config(callbacks_cfg[k]) for k in callbacks_cfg]

    trainer = Trainer.from_argparse_args(trainer_opt, **trainer_kwargs)
    trainer.logdir = logdir  ###

    # data
    data = instantiate_from_config(config.data)
    # NOTE according to https://pytorch-lightning.readthedocs.io/en/latest/datamodules.html
    # calling these ourselves should not be necessary but it is.
    # lightning still takes care of proper multiprocessing though
    data.prepare_data()
    data.setup()
    print("#### Data #####")
    for k in data.datasets:
        print(f"{k}, {data.datasets[k].__class__.__name__}, {len(data.datasets[k])}")

    # configure learning rate
    bs, base_lr = config.data.params.batch_size, config.model.base_learning_rate
    if not cpu:
        ngpu = len(lightning_config.trainer.gpus.strip(",").split(','))
    else:
        ngpu = 1
    if 'accumulate_grad_batches' in lightning_config.trainer:
        accumulate_grad_batches = lightning_config.trainer.accumulate_grad_batches
    else:
        accumulate_grad_batches = 1
    print(f"accumulate_grad_batches = {accumulate_grad_batches}")
    lightning_config.trainer.accumulate_grad_batches = accumulate_grad_batches
    if opt.scale_lr:
        model.learning_rate = accumulate_grad_batches * ngpu * bs * base_lr
        print(
            "Setting learning rate to {:.2e} = {} (accumulate_grad_batches) * {} (num_gpus) * {} (batchsize) * {:.2e} (base_lr)".format(
                model.learning_rate, accumulate_grad_batches, ngpu, bs, base_lr))
    else:
        model.learning_rate = base_lr
        print("++++ NOT USING LR SCALING ++++")
        print(f"Setting learning rate to {model.learning_rate:.2e}")


    # allow checkpointing via USR1
    def melk(*args, **kwargs):
        # run all checkpoint hooks
        if trainer.global_rank == 0:
            print("Summoning checkpoint.")
            ckpt_path = os.path.join(ckptdir, "last.ckpt")
            trainer.save_checkpoint(ckpt_path)


    def divein(*args, **kwargs):
        if trainer.global_rank == 0:
            import pudb;
            pudb.set_trace()


    import signal

    signal.signal(signal.SIGUSR1, melk)
    signal.signal(signal.SIGUSR2, divein)

    # run
    if opt.train:
        try:
            trainer.fit(model, data)
        except Exception:
            melk()
            raise
    if not opt.no_test and not trainer.interrupted:
        trainer.test(model, data)
except Exception:
    if opt.debug and trainer.global_rank == 0:
        try:
            import pudb as debugger
        except ImportError:
            import pdb as debugger
        debugger.post_mortem()
    raise
finally:
    # move newly created debug project to debug_runs
    if opt.debug and not opt.resume and trainer.global_rank == 0:
        dst, name = os.path.split(logdir)
        dst = os.path.join(dst, "debug_runs", name)
        os.makedirs(os.path.split(dst)[0], exist_ok=True)
        os.rename(logdir, dst)
    if trainer.global_rank == 0:
        print(trainer.profiler.summary())

INFO:pytorch_lightning.utilities.seed:Global seed set to 23


NameError: ignored